<a href="https://colab.research.google.com/github/kimdonggyu2008/combined/blob/main/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#api활용할때의 통합 사용

#파이프라인 테스트


In [1]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import re
import math


In [2]:

# 요약 모델 및 토크나이저 설정
summarizer_model_name = "/content/drive/MyDrive/summarizer/model/model_part_2"
summarizer_tokenizer = PegasusTokenizer.from_pretrained(summarizer_model_name)
summarizer_model = PegasusForConditionalGeneration.from_pretrained(summarizer_model_name)


In [3]:

# 번역 모델 및 토크나이저 설정
translator_model_name = "/content/drive/MyDrive/translator/model/model_part_3"
translator_tokenizer = AutoTokenizer.from_pretrained(translator_model_name)
translator_model = AutoModelForSeq2SeqLM.from_pretrained(translator_model_name)


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [4]:

# def summarize_article(article_text):
#     # 입력 텍스트를 요약
#     inputs = summarizer_tokenizer(article_text, max_length=1024, return_tensors="pt", truncation=True)
#     summary_ids = summarizer_model.generate(inputs["input_ids"], max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
#     summary = summarizer_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#     return summary


In [14]:

def translate_text(text):
    # 요약된 텍스트를 번역
    inputs = translator_tokenizer(text, return_tensors="pt", truncation=True)
    translated_ids = translator_model.generate(inputs["input_ids"], max_length=512, num_beams=4, early_stopping=True)
    translation = translator_tokenizer.decode(translated_ids[0], skip_special_tokens=True)
    return translation


In [6]:

# def summarize_and_translate(article_text):
#     # 외신 기사 요약 및 번역 파이프라인
#     summary = summarize_article(article_text)
#     translation = translate_text(summary)
#     return translation


In [7]:
def split_text_with_last_sentence_overlap(text, target_chunk_length=2048):
    # 문장 단위로 텍스트 분할
    sentences = re.split(r'(?<=[.!?]) +', text)
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        # 현재 문장을 청크에 추가
        if len(current_chunk) + len(sentence) <= target_chunk_length:
            current_chunk += sentence + " "
        else:
            # 청크에 문장 추가 후, 마지막 문장을 다음 청크에 포함
            chunks.append(current_chunk.strip())
            current_chunk = chunks[-1].split()[-1] + " " + sentence + " "

    # 마지막 청크 추가
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

In [8]:
def summarize_long_text(article_text, target_chunk_length=2048):
    # 텍스트를 분할하여 요약
    chunks = split_text_with_last_sentence_overlap(article_text, target_chunk_length)
    summaries = []

    for chunk in chunks:
        #print(len(chunk))
        #print(chunk)
        #print("현재 청크")
        inputs = summarizer_tokenizer(chunk, max_length=target_chunk_length, return_tensors="pt", truncation=True)
        summary_ids = summarizer_model.generate(inputs["input_ids"], max_length=200, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
        summary = summarizer_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)

    return summaries

In [9]:
def translate_and_combine_summaries(summaries):
    # 번역된 텍스트를 담을 리스트
    translated_summaries = []

    for summary in summaries:
        translation = translate_text(summary)
        translated_summaries.append(translation)

    # 번역된 요약 문장들을 하나로 합치기
    combined_translation = "\n".join(translated_summaries)
    return combined_translation


#번역할 기사

In [10]:
article_text = """
Japanese voters are today heading to the polls in a snap election, following a tumultuous few years for the ruling party which saw a “cascade” of scandals, widespread voter apathy and record-low approval ratings.

The election was announced by Shigeru Ishiba three days after he was selected as the leader of the ruling Liberal Democratic Party (LDP) - before he had been officially sworn in as prime minister.

The decision was made despite the LDP seeing approval ratings of below 20% earlier in the year, in the wake of a political fundraising corruption scandal.

Yet the LDP still remains the strongest contender against opposition parties which have failed to unite, or convince voters they are a viable option to govern.

The main opposition party had an approval rating of just 6.6% before parliament was dissolved.

“It is so hard to make decisions to choose parties, I think people are losing interest,” Miyuki Fujisaki, a long-time LDP supporter who works in the care-home sector, told the BBC ahead of polls opening.

The LDP, she said, has its problems with alleged corruption, “but the opposition also does not stand out at all”.

“They sure complain a lot, but it’s not at all clear on what they want to do,” the 66-year-old said.

For all the apathy, politics in Japan has been moving at a fast pace in recent months.

Shigeru Ishiba took over as prime minister after being voted in by the ruling party following his predecessor Fumio Kishida - who had been in the role since 2021 – making a surprise decision to step down in August.

The move to call the election came at a time when the LDP is desperate to restore its tarnished image among the public.


Ishiba - a long-time politician who previously served as defence minister - has described it as the “people’s verdict”.

But whether it’s enough to restore trust in the LDP - which has been in power almost continuously since 1955 – is uncertain.

A series of scandals has tarnished the ruling party’s reputation. Chief among them is the party’s relationship with the controversial Unification Church - described by critics as a “cult” - and the level of influence it had on lawmakers.

Then came the revelations of the political funding corruption scandal. Japan’s prosecutors have been investigating dozens of LDP lawmakers accused of pocketing proceeds from political fundraising events. Those allegations - running into the millions of dollars - led to the dissolution of powerful factions, the backbone of its internal party politics.

“What a wretched state the ruling party is in,” said Michiko Hamada, who had travelled to Urawa station, on outskirts of Tokyo, for an opposition campaign rally.

“That is what I feel most. It is tax evasion and it’s unforgivable.”

It strikes her as particularly egregious at a time when people in Japan are struggling with high prices. Wages have not changed for three decades – dubbed “the lost 30 years” – but prices have risen at the fastest rate in nearly half a century in the last two years.

This month, as voters were getting ready to go to the polls, saw more price hikes on thousands of food products as well as other day-to-day provisions like mail, pharmaceuticals, electricity and gas.

“I pay 10,000 yen or 20,000 yen ($65 - $130; £50 - £100) more for the food per month (than I used to),” Ms Hamada said.

“And I’m not buying the things I used to buy. I am trying to save up but it still costs more.



Things like fruit are very expensive.”

She is not the only one concerned with high prices. Pensioner Chie Shimizu says she now must work part-time to make ends meet.

“Our hourly wage has gone up a bit but it does not match the prices,” she told the BBC as she picked up some food from a stand at Urawa station. “I come to places like this to find something cheaper and good because everything in regular shops is expensive.”

Ms Shimizu has not voted for years but might this time - although she is not sure which candidate or which party to vote for.

“I can't find anyone that I want to vote for. I feel like there's no one who I can trust to be our leader. I wonder about those who become an MP for their own greed.”

Against this backdrop, it might look like Ishiba has taken a political gamble. His party had held 247 of 465 seats in the lower house, while its coalition partner, Komeito, had 32. A party needs 233 seats to control the house, known as the Diet.

There are now fears Komeito may fail to reach that number again, while the main opposition - which had 98 seats in the previous parliament - began to pick up momentum with voters ahead of Sunday’s election.

“I think the LDP has dug itself a very deep hole to climb out of. It does not enjoy public trust, and why should it?


There's just been a cascade of scandals,” Jeff Kingston, professor of Asian studies and history at Temple University Japan, told the BBC.

But he does not think any of this will necessarily lead to the party losing the elections.

“I think they (LDP) are worried they're going to lose some marginal seats, and there are questions hanging over Komeito and how effective a coalition partner they will be,” Prof Kingston said.

Should they pull off a win, Miyuki Fujisaki, the care-home sector worker, warns they will have to do more than just pay lip service to change.

“I want them to show us what they are going to do so this [the scandals] doesn’t happen again,” she said. “They need to prove it - not just say it like they often do in the election time.”


"""

#실행 코드


In [11]:
print(len(article_text))

5491


In [12]:
summaries=summarize_long_text(article_text)
print(summaries)

['Japanese voters are heading to the polls in a snap election. The ruling Liberal Democratic Party (LDP) saw its approval ratings fall below 20% earlier in the year. The main opposition party had an approval rating of just 6.6% before parliament was dissolved. The LDP is desperate to restore its tarnished image.', "Japan's ruling party has been hit by a series of scandals in recent months. One of the biggest is the political funding corruption scandal. But people in Japan are struggling with high prices. Pensioner Chie Shimizu says she now must work part-time to make ends meet.", 'Voters in Japan go to the polls on Sunday to elect a new parliament. The ruling Liberal Democratic Party (LDP) had held 247 of 465 seats in the lower house, while its coalition partner, Komeito, had 32. A party needs 233 seats to control the house, known as the Diet.']


In [28]:
combined_translation = translate_and_combine_summaries(summaries)
print(combined_translation)

(LDP).
The's.
Thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thers of thes of thes of thers of thers of thes of thers of thes of thers of thes of thers of thes of thes of thes of thes of thers of thes of thes of thers of thes of thes of thes.


In [ ]:
#헬싱키 모델
for summary in summaries:
    translation = translator_tokenizer(summary,return_tensors="pt", truncation=True)
    print(summary)
    print(translation)

#토크나이징 테스트


In [25]:
from transformers import MarianMTModel, MarianTokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

translator_tokenizer2 = AutoTokenizer.from_pretrained("facebook/nllb-200-3.3B")
translator_model2 = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-3.3B")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model-00001-of-00003.bin:  20%|#9        | 1.38G/6.93G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/8.55G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/2.10G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [26]:

for summary in summaries:
    #translation = translator_tokenizer(summary,return_tensors="pt", truncation=True)
    translation = translator_tokenizer2(summary,return_tensors="pt", truncation=True)
    #translated_summaries.append(translation)
    print(summary)
    print(translation)

Japanese voters are heading to the polls in a snap election. The ruling Liberal Democratic Party (LDP) saw its approval ratings fall below 20% earlier in the year. The main opposition party had an approval rating of just 6.6% before parliament was dissolved. The LDP is desperate to restore its tarnished image.
{'input_ids': tensor([[256047, 134724,  12182,   1044,   2442, 239353,    202,    349,     17,
         127410,    108,      9, 137288, 171600, 248075,   1617,     51,  89315,
         225116, 177164,  54261,    104, 248185,  21816, 248161,  25468,   6629,
         203065,   5456,   2895,  22330, 109759,  26562, 149212,    108,    349,
          17174, 248075,   1617,   8385, 193165,  38559,   2908,    111, 203065,
         127135,    452,   7027,   8267,  45856,  19553,    413,  63019,   1398,
            889,  61227,   6341, 248075,   1617,    155,  21816,    248,  81328,
            502,    202,   7640,    478,   6629,   1609, 248063,  41120,  52101,
         248075,      2]])

In [27]:

def translate_text2(text):
    # 요약된 텍스트를 번역
    inputs = translator_tokenizer2(text, return_tensors="pt", truncation=True)
    translated_ids = translator_model2.generate(inputs["input_ids"], max_length=512, num_beams=4, early_stopping=True)
    translation = translator_tokenizer2.decode(translated_ids[0], skip_special_tokens=True)
    return translation


In [29]:
def translate_and_combine_summaries2(summaries):
    # 번역된 텍스트를 담을 리스트
    translated_summaries = []

    for summary in summaries:
        translation = translate_text2(summary)
        translated_summaries.append(translation)

    # 번역된 요약 문장들을 하나로 합치기
    combined_translation = "\n".join(translated_summaries)
    return combined_translation

In [30]:
combined_translation = translate_and_combine_summaries2(summaries)
print(combined_translation)

KeyboardInterrupt: 

#후보 3


In [31]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

translator_tokenizer3 = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
translator_model3 = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [61]:
from transformers import pipeline

translator = pipeline('translation', model='facebook/nllb-200-distilled-600M', device=0, src_lang='eng_Latn', tgt_lang='kor_Hang', max_length=512)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [62]:
translated_summaries=[]
for summary in summaries:
        translation = translator(summary, max_length=512)
        translated_summaries.append(translation)

In [63]:
print(translated_summaries)

[[{'translation_text': '일본 유권자들은 조기 선거에서 투표에 향하고 있다. 집권자 자유민주당 (LDP) 은 올해 초 지지율이 20% 이하로 떨어졌다. 주요 야당은 의회 해체되기 전 6.6%의 지지율을 기록했다. LDP는 망가진 이미지를 회복하기 위해 절실히 노력하고 있다.'}], [{'translation_text': '일본 집권당은 최근 몇 달 동안 일련의 스캔들에 의해 타격을 받았습니다. 가장 큰 스캔들 중 하나는 정치 자금 부패 스캔들입니다. 하지만 일본 사람들은 높은 가격에 어려움을 겪고 있습니다. 은퇴자 치 시미즈우는 이제 일정을 이루기 위해 파트 타임으로 일해야 한다고 말합니다.'}], [{'translation_text': '일본의 유권자들은 일요일 새 의회를 선출하기 위해 투표에 나간다. 집권 리버럴 민주당 (LDP) 은 하원 465석 중 247석을 차지했고, 연립 파트너인 코메이토는 32석을 차지했다.'}]]


#모델 hugging face 업로드

In [ ]:
# !pip install transformers huggingface_hub

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()  # API 토큰을 입력하면 됩니다.


In [ ]:
# from huggingface_hub import HfApi, HfFolder, Repository

# summarizer_model_repo = "EXP442/pegasus_summarizer"
# translator_model_repo = "EXP442/seq2seq_translator"

# # Summarizer 모델 업로드
# api = HfApi()
# api.upload_folder(folder_path=summarizer_model_name, repo_id=summarizer_model_repo)

# # Translator 모델 업로드
# api.upload_folder(folder_path=translator_model_name, repo_id=translator_model_repo)